In [1]:
from tool.functions import *

In [2]:
download_all_bidixes()

2018-06-11 22:02:17,226 | INFO : Start
2018-06-11 22:13:57,475 | INFO : Finish


For now I delete ces-ces, ita-ita, eus-eu_bis, hbs_HR-hbs_SR, zh_CN-zh_TW

In [2]:
%time monodix()

2018-06-12 16:41:10,577 | INFO : started
2018-06-12 16:44:19,892 | INFO : finished
Wall time: 3min 9s


In [1]:
from tool.functions import get_relevant_languages

In [2]:
%time get_relevant_languages('spa', 'eng')

Wall time: 44 ms


In [1]:
from tool.functions import load_file, change_encoding, built_from_file

In [2]:
%time load_file('spa', 'eng')

2018-06-11 22:32:06,560 | INFO : cat-ita started
2018-06-11 22:32:39,285 | INFO : eng-cat started
2018-06-11 22:37:19,892 | INFO : eng-ita started
2018-06-11 22:38:54,490 | INFO : eng-spa started
2018-06-11 22:41:22,105 | INFO : fra-cat started
2018-06-11 22:45:20,731 | INFO : fra-eng started
2018-06-11 22:46:16,861 | INFO : fra-ita started
2018-06-11 22:46:48,962 | INFO : fra-spa started
2018-06-11 22:48:43,558 | INFO : spa-cat started
2018-06-11 22:53:54,753 | INFO : spa-ita started
Wall time: 22min 55s


In [3]:
%time change_encoding('spa-eng')

Wall time: 296 ms


In [77]:
%time G = built_from_file('spa-eng')

Wall time: 26.8 s


In [5]:
len(G.edges())

538634

In [6]:
len(G.nodes())

285935

** Compress data (tags) **

In [66]:
def compress_file(filename):
    with open (filename, 'r', encoding='utf-8') as f:
        with open (filename+'-short', 'w', encoding='utf-8') as outp:
            for side, word1, word2 in nodes_from_file(filename):
                string = str(side) + '\t' + compress(word1).write(mode='bi') + '\t' + compress(word2).write(mode='bi') + '\n'
                outp.write(string)        

In [67]:
%time compress_file('spa-eng')

Wall time: 5.69 s


In [65]:
def compress(word):
    s = list(sorted(word.s, key=len))
    length = len(s[0]) - 1
    if length == -1: print('Alarm!\t', word)
    s2 = []
    for key, value in enumerate(s[-1]):
        if key > length: s2.append('('+value+')')
        else: s2.append(value)
    word.s = [Tags(s2)]
    return word

In [68]:
word = Word('capital','spa',[Tags(['n','f']), Tags('n'), Tags(['n','f','sg'])])

In [69]:
compress(word)

spa$capital$[n-(f)-(sg)]

Compression turned out to be not efficient

** Search **

https://networkx.github.io/documentation/networkx-2.1/_modules/networkx/algorithms/shortest_paths/unweighted.html#single_source_shortest_path

In [114]:
from itertools import islice

In [119]:
def _single_shortest_path_length(adj, firstlevel, cutoff, lang):
    """ Variant of NetworkX function _single_shortest_path_length"""
    seen = {}                  # level (number of hops) when seen in BFS
    level = 0                  # the current level
    nextlevel = firstlevel     # dict of nodes to check at next level

    while nextlevel and cutoff >= level:
        thislevel = nextlevel  # advance to next level
        nextlevel = {}         # and start a new list (fringe)
        for v in thislevel:
            if v not in seen:
                seen[v] = level  # set the level of vertex v
                #if v.lang != lang:
                nextlevel.update(adj[v])  # add neighbors of v
                if v.lang == lang:
                    #yield (v, level)
                    yield v
        level += 1
    del seen

def possible_translations(G, source, lang, cutoff=None):
    """ Variant of NetworkX function single_source_shortest_path_length"""
    if source not in G: raise nx.NodeNotFound('Source {} is not in G'.format(source))
    if cutoff is None: cutoff = float('inf')
    nextlevel = {source: 1}
    return list(islice(_single_shortest_path_length(G.adj, nextlevel, cutoff, lang), 30))

In [121]:
%timeit s = possible_translations(G, Word('be','eng',[Tags(['vblex']),Tags(['vblex','pp'])]), 'spa', cutoff=40)

4.08 ms ± 156 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [124]:
len(possible_translations(G, Word('be','eng',[Tags(['vblex']),Tags(['vblex','pp'])]), 'spa', cutoff=40))

30

In [96]:
%timeit s = nx.single_source_shortest_path_length(G, Word('be','eng',[Tags(['vblex']),Tags(['vblex','pp'])]),cutoff=40)

390 ms ± 45.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
